In [0]:
dbutils.widgets.text('marketplace', '')
marketplace = dbutils.widgets.get('marketplace')

# IAM User : ebzspapi_new
# IAM Role : ebzsellingapiRole
# AVC App Client : ebzspapiApp_new
A_KEY = 'AKIA........2GE'
A_SECRET = 'Kz54x6P............zb20vBx'
ARN = 'arn:aws:iam::4668...4117:role/ebzsellingapiRole'


C_KEY = 'amzn1.application-oa2-client.c895.........1cee69f'
C_SECRET_KEY = '7122788088f86.............ed55a46827d1d09c76'
C_REFRESH_TOKEN = 'Atzr|IwEBIDL5-mE5P_.......ypkXhs'


if marketplace == 'DE':
    MarketplaceID = 'A1PA6795UKMFR9'
    
elif  marketplace == 'IT':
    MarketplaceID = 'APJ6JRA9NG5V4'
    
elif marketplace == 'FR':
    MarketplaceID = 'A13V1IB3VIYZZH'
    
elif marketplace == 'NL':
     MarketplaceID = 'A1805IZSGTT6HS'
        
elif marketplace == 'ES':
     MarketplaceID = 'A1RKKUPIHCS9HS'
        
elif marketplace == 'UK':
     MarketplaceID = 'A1F83G8C2ARO7P'   
        
elif marketplace == 'SE':
     MarketplaceID = 'A2NODRKZP88ZB9'   

elif marketplace == 'PL':
     MarketplaceID = 'A1C3SOZRARQ6R3'   

In [0]:
from time import sleep
import requests
import re
import json
import datetime
import gzip
from requests_aws4auth import AWS4Auth


def get_datetime():
    return datetime.datetime.utcnow().strftime("%Y%m%dT%H%M%SZ")


def get_access_token():
    global C_KEY, C_SECRET_KEY, C_REFRESH_TOKEN
    r = requests.post("https://api.amazon.com/auth/o2/token",
                      data={
                            "grant_type":"refresh_token",
                            "refresh_token":C_REFRESH_TOKEN,
                            "client_id": C_KEY,
                            "client_secret": C_SECRET_KEY
                            })

    return r.json()['access_token']



def assume_role():
    r = requests.get(f"https://sts.amazonaws.com?Version=2011-06-15&Action=AssumeRole&RoleSessionName=Test&RoleArn={ARN}&DurationSeconds=3600", 
                      auth = AWS4Auth(A_KEY, A_SECRET, 'us-east-1', 'sts')
                    )

    return (re.findall('<AccessKeyId>(.+)</AccessKeyId>', r.text)[0], re.findall('<SecretAccessKey>(.+)</SecretAccessKey>', r.text)[0], re.findall('<SessionToken>(.+)</SessionToken>', r.text)[0])


access_token = get_access_token()
access_key, secret_key, session_token = assume_role()


In [0]:
# Europe endpoint : https://sellingpartnerapi-eu.amazon.com
# Europe AWS region : eu-west-1

# This request needs parameters. Use the 'params' in the request.
# For requests that needs body information, use 'data' in the request.


url = "https://sellingpartnerapi-eu.amazon.com/fba/inventory/v1/summaries"

headers= {
            'x-amz-access-token': access_token,
            'Content-Type': 'application/json'
         }

params = {
            'details'          : 'true',
            'granularityType'  : 'Marketplace',
            'granularityId'    :  MarketplaceID,
            'marketplaceIds'   :  MarketplaceID
    
          }

auth = AWS4Auth(access_key, secret_key, 'eu-west-1', 'execute-api', session_token=session_token)

response = requests.get(url = url, headers = headers, params = params, auth = auth)
jsonfile = response.json()
file = json.dumps(jsonfile)

In [0]:
containerName = "amazon-inventory-sira"
storageAccountName = "blobinventory"
sas = "sp=racwdli&st=2022-08-10T2.................5JY%3D"
config = "fs.azure.sas." + containerName+ "." + storageAccountName + ".blob.core.windows.net"

try:
    dbutils.fs.unmount("/mnt/amazon-inventory-sira")
except:
    pass
    
try:
    dbutils.fs.mount(
      source = f"wasbs://{containerName}@{storageAccountName}.blob.core.windows.net",
      mount_point = "/mnt/amazon-inventory-sira",
      extra_configs = {config : sas})
except:
    pass

/mnt/amazon-inventory-sira has been unmounted.

In [0]:
dbutils.fs.put('dbfs:/mnt/amazon-inventory-sira/' + marketplace + '.json' , file)

Wrote 58871 bytes.
Out[33]: True